<a href="https://colab.research.google.com/github/armen1s/BOT/blob/main/Titan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ======================================================
# 🏗 МОДУЛЬ 1: MASTER DATA ENGINE (v1.1 - FIXED)
# ======================================================
import os
import time
import warnings
from datetime import datetime

# 0. УСТАНОВКА
print("🛠 Подготовка инструментов...")
os.system("pip install ccxt pandas numpy numba -q")

import ccxt
import pandas as pd
import numpy as np
from numba import jit

# 1. НАСТРОЙКИ
SYMBOL = 'BTC/USD'
TIMEFRAME = '1h'
START_DATE = '2025-01-01 00:00:00'
FILENAME = 'titan_raw_history.csv'
FINAL_FILE = 'titan_final_data.csv'

exchange = ccxt.bitstamp({'enableRateLimit': True})

# 2. СИНХРОНИЗАЦИЯ (С ЗАЩИТОЙ ОТ ОБРЫВОВ)
def sync_data():
    print(f"🚀 Синхронизация {SYMBOL}...")

    if not os.path.exists(FILENAME):
        since_ts = int(pd.Timestamp(START_DATE).timestamp() * 1000)
        with open(FILENAME, 'w') as f:
            f.write("timestamp,open,high,low,close,volume\n")
    else:
        try:
            df_check = pd.read_csv(FILENAME)
            if not df_check.empty:
                since_ts = int(df_check.iloc[-1]['timestamp']) + 1
            else:
                since_ts = int(pd.Timestamp(START_DATE).timestamp() * 1000)
        except:
            since_ts = int(pd.Timestamp(START_DATE).timestamp() * 1000)

    while True:
        try:
            batch = exchange.fetch_ohlcv(SYMBOL, TIMEFRAME, since=since_ts, limit=1000)
            if not batch: break

            with open(FILENAME, 'a') as f:
                for c in batch:
                    f.write(f"{c[0]},{c[1]},{c[2]},{c[3]},{c[4]},{c[5]}\n")

            last_ts = batch[-1][0]
            if last_ts >= exchange.milliseconds() - 3600000:
                print("   ✅ История актуальна.")
                break

            since_ts = last_ts + 1
            print(".", end="")
            time.sleep(0.1)

        except Exception as e:
            print(f"\n⚠️ Ошибка сети: {e}. Ждем..."); time.sleep(5)

# 3. МАТЕМАТИКА (Numba JIT)
@jit(nopython=True)
def calculate_fdi(prices, window=48):
    n = len(prices)
    res = np.full(n, np.nan)
    for i in range(window, n):
        seg = prices[i-window:i]
        mx, mn = np.max(seg), np.min(seg)
        if mx == mn: res[i] = 1.0; continue
        norm = (seg - mn) / (mx - mn)
        length = np.sum(np.sqrt((1/(window-1))**2 + np.diff(norm)**2))
        res[i] = 1 + (np.log(length) + np.log(2)) / np.log(2 * (window - 1))
    return res

def apply_forensics(df):
    df_c = df.copy()
    vol = df_c['close'].pct_change().rolling(24).std()
    body_max = np.maximum(df_c['open'], df_c['close'])
    expected = df_c['close'] * vol
    anomalies = (df_c['high'] - body_max) / expected.replace(0, np.nan) > 4.0
    if anomalies.sum() > 0:
        print(f"   🧹 Forensics: Исправлено {anomalies.sum()} сбоев цены.")
        df_c.loc[anomalies, 'high'] = body_max[anomalies] + (3.0 * expected[anomalies])
    return df_c

# 4. КОНВЕЙЕР ОБРАБОТКИ
sync_data()
print("⚙️ Feature Engineering...")

df = pd.read_csv(FILENAME)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df = df.set_index('timestamp')
df = df[~df.index.duplicated(keep='first')]

# [ВАЖНО!] ЛЕЧЕНИЕ ВРЕМЕНИ (Time Grid Repair)
# Если были пропуски (биржа лежала), мы их заполняем, чтобы сохранить ритм
full_idx = pd.date_range(start=df.index.min(), end=df.index.max(), freq='h')
df = df.reindex(full_idx)
# Заполняем пустоты методом "Forward Fill" (цена не менялась, пока биржа лежала)
df = df.ffill()

# Forensics
df = apply_forensics(df)

# Логарифмы
df['Close_Log'] = np.log(df['close'])
df['Volume_Log'] = np.log1p(df['volume'])

# Физика
df['FDI'] = calculate_fdi(df['close'].values)

# Yang-Zhang
w = 24
ho = np.log(df['high'] / df['open'])
lo = np.log(df['low'] / df['open'])
co = np.log(df['close'] / df['open'])
oc = np.log(df['open'] / df['close'].shift(1))
yz_var = oc.rolling(w).var() + 0.134*co.rolling(w).var() + 0.866*((ho*(ho-co)) + (lo*(lo-co))).rolling(w).mean()
df['YZ_Vol'] = np.sqrt(yz_var).fillna(0)
df['VPT'] = (df['Volume_Log'] * df['Close_Log'].diff()).cumsum().fillna(0)

# Структура для TFT
df = df.sort_index()
df['time_idx'] = range(len(df)) # Теперь это идеальная последовательность
df['group_id'] = 'BTC'
df['hour'] = df.index.hour.astype(str).astype("category")
df['day'] = df.index.dayofweek.astype(str).astype("category")

df.dropna(inplace=True)
df.to_csv(FINAL_FILE)

print("\n" + "="*50)
print(f"✅ DATA ENGINE ЗАВЕРШЕН.")
print(f"   Баров: {len(df)}")
print(f"   Пропусков времени: 0 (Grid Repaired)")
print(f"📂 Файл готов: {FINAL_FILE}")
print("="*50)

🛠 Подготовка инструментов...
🚀 Синхронизация BTC/USD...
⚙️ Feature Engineering...
   🧹 Forensics: Исправлено 1 сбоев цены.

✅ DATA ENGINE ЗАВЕРШЕН.
   Баров: 9256
   Пропусков времени: 0 (Grid Repaired)
📂 Файл готов: titan_final_data.csv


In [3]:

# ======================================================
# 🏗 МОДУЛЬ 1: MASTER DATA ENGINE (v1.1 - FIXED)
# ======================================================
import os
import time
import warnings
from datetime import datetime

# 0. УСТАНОВКА
print("🛠 Подготовка инструментов...")
os.system("pip install ccxt pandas numpy numba -q")

import ccxt
import pandas as pd
import numpy as np
from numba import jit

# 1. НАСТРОЙКИ
SYMBOL = 'BTC/USD'
TIMEFRAME = '1h'
START_DATE = '2025-01-01 00:00:00'
FILENAME = 'titan_raw_history.csv'
FINAL_FILE = 'titan_final_data.csv'

exchange = ccxt.bitstamp({'enableRateLimit': True})

# 2. СИНХРОНИЗАЦИЯ (С ЗАЩИТОЙ ОТ ОБРЫВОВ)
def sync_data():
    print(f"🚀 Синхронизация {SYMBOL}...")

    if not os.path.exists(FILENAME):
        since_ts = int(pd.Timestamp(START_DATE).timestamp() * 1000)
        with open(FILENAME, 'w') as f:
            f.write("timestamp,open,high,low,close,volume\n")
    else:
        try:
            df_check = pd.read_csv(FILENAME)
            if not df_check.empty:
                since_ts = int(df_check.iloc[-1]['timestamp']) + 1
            else:
                since_ts = int(pd.Timestamp(START_DATE).timestamp() * 1000)
        except:
            since_ts = int(pd.Timestamp(START_DATE).timestamp() * 1000)

    while True:
        try:
            batch = exchange.fetch_ohlcv(SYMBOL, TIMEFRAME, since=since_ts, limit=1000)
            if not batch: break

            with open(FILENAME, 'a') as f:
                for c in batch:
                    f.write(f"{c[0]},{c[1]},{c[2]},{c[3]},{c[4]},{c[5]}\n")

            last_ts = batch[-1][0]
            if last_ts >= exchange.milliseconds() - 3600000:
                print("   ✅ История актуальна.")
                break

            since_ts = last_ts + 1
            print(".", end="")
            time.sleep(0.1)

        except Exception as e:
            print(f"\n⚠️ Ошибка сети: {e}. Ждем..."); time.sleep(5)

# 3. МАТЕМАТИКА (Numba JIT)
@jit(nopython=True)
def calculate_fdi(prices, window=48):
    n = len(prices)
    res = np.full(n, np.nan)
    for i in range(window, n):
        seg = prices[i-window:i]
        mx, mn = np.max(seg), np.min(seg)
        if mx == mn: res[i] = 1.0; continue
        norm = (seg - mn) / (mx - mn)
        length = np.sum(np.sqrt((1/(window-1))**2 + np.diff(norm)**2))
        res[i] = 1 + (np.log(length) + np.log(2)) / np.log(2 * (window - 1))
    return res

def apply_forensics(df):
    df_c = df.copy()
    vol = df_c['close'].pct_change().rolling(24).std()
    body_max = np.maximum(df_c['open'], df_c['close'])
    expected = df_c['close'] * vol
    anomalies = (df_c['high'] - body_max) / expected.replace(0, np.nan) > 4.0
    if anomalies.sum() > 0:
        print(f"   🧹 Forensics: Исправлено {anomalies.sum()} сбоев цены.")
        df_c.loc[anomalies, 'high'] = body_max[anomalies] + (3.0 * expected[anomalies])
    return df_c

# 4. КОНВЕЙЕР ОБРАБОТКИ
sync_data()
print("⚙️ Feature Engineering...")

df = pd.read_csv(FILENAME)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df = df.set_index('timestamp')
df = df[~df.index.duplicated(keep='first')]

# [ВАЖНО!] ЛЕЧЕНИЕ ВРЕМЕНИ (Time Grid Repair)
# Если были пропуски (биржа лежала), мы их заполняем, чтобы сохранить ритм
full_idx = pd.date_range(start=df.index.min(), end=df.index.max(), freq='h')
df = df.reindex(full_idx)
# Заполняем пустоты методом "Forward Fill" (цена не менялась, пока биржа лежала)
df = df.ffill()

# Forensics
df = apply_forensics(df)

# Логарифмы
df['Close_Log'] = np.log(df['close'])
df['Volume_Log'] = np.log1p(df['volume'])

# Физика
df['FDI'] = calculate_fdi(df['close'].values)

# Yang-Zhang
w = 24
ho = np.log(df['high'] / df['open'])
lo = np.log(df['low'] / df['open'])
co = np.log(df['close'] / df['open'])
oc = np.log(df['open'] / df['close'].shift(1))
yz_var = oc.rolling(w).var() + 0.134*co.rolling(w).var() + 0.866*((ho*(ho-co)) + (lo*(lo-co))).rolling(w).mean()
df['YZ_Vol'] = np.sqrt(yz_var).fillna(0)
df['VPT'] = (df['Volume_Log'] * df['Close_Log'].diff()).cumsum().fillna(0)

# Структура для TFT
df = df.sort_index()
df['time_idx'] = range(len(df)) # Теперь это идеальная последовательность
df['group_id'] = 'BTC'
df['hour'] = df.index.hour.astype(str).astype("category")
df['day'] = df.index.dayofweek.astype(str).astype("category")

df.dropna(inplace=True)
df.to_csv(FINAL_FILE)

print("\n" + "="*50)
print(f"✅ DATA ENGINE ЗАВЕРШЕН.")
print(f"   Баров: {len(df)}")
print(f"   Пропусков времени: 0 (Grid Repaired)")
print(f"📂 Файл готов: {FINAL_FILE}")
print("="*50)

🛠 Подготовка инструментов...
🚀 Синхронизация BTC/USD...
⚙️ Feature Engineering...
   🧹 Forensics: Исправлено 1 сбоев цены.

✅ DATA ENGINE ЗАВЕРШЕН.
   Баров: 9256
   Пропусков времени: 0 (Grid Repaired)
📂 Файл готов: titan_final_data.csv


In [5]:
# ======================================================
# 🧠 МОДУЛЬ 2: BRAIN + BLIND SIMULATION (ИСПРАВЛЕННЫЙ)
# ======================================================
import os
import warnings

# --- 0. АВТО-УСТАНОВКА БИБЛИОТЕК ---
print("🛠 Установка AI-библиотек (Lightning, Forecasting)...")
# Эта строчка исправит ошибку ModuleNotFoundError
os.system("pip install lightning pytorch_forecasting -q")

import numpy as np
import pandas as pd
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss
import shutil

# --- 1. НАСТРОЙКИ ---
print("⚙️ Инициализация TITAN AI...")
pl.seed_everything(42)
warnings.filterwarnings("ignore")

# Загрузка данных из Модуля 1
data_path = 'titan_final_data.csv'
if not os.path.exists(data_path):
    raise FileNotFoundError("❌ ОШИБКА: Файл 'titan_final_data.csv' не найден. Сначала выполните Модуль 1!")

df = pd.read_csv(data_path)
# Исправление типов данных
df['group_id'] = df['group_id'].astype(str)
df['hour'] = df['hour'].astype(str)
df['day'] = df['day'].astype(str)
df['time_idx'] = df['time_idx'].astype(int)

# --- 2. ХИРУРГИЯ: ОТРЕЗАЕМ ЯНВАРЬ (СЛЕПОЙ ТЕСТ) ---
BLIND_TEST_HOURS = 720  # 30 дней
cutoff_idx = df["time_idx"].max() - BLIND_TEST_HOURS

print(f"📉 Всего истории: {len(df)} часов")
print(f"✂️ Обучение (Train): До индекса {cutoff_idx} (Прошлое)")
print(f"🔒 Тест (Blind):     Последние {BLIND_TEST_HOURS} часов (Скрытое будущее)")

# --- 3. ПОДГОТОВКА ДАТАСЕТА ---
training_cutoff = cutoff_idx - 12

training_dataset = TimeSeriesDataSet(
    df[lambda x: x.time_idx <= training_cutoff], # Обучаем только на прошлом
    time_idx="time_idx",
    target="Close_Log",
    group_ids=["group_id"],
    min_encoder_length=48, max_encoder_length=48,
    min_prediction_length=12, max_prediction_length=12,
    static_categoricals=["group_id"],
    time_varying_known_categoricals=["hour", "day"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["Close_Log", "Volume_Log", "YZ_Vol", "FDI", "VPT"],
    target_normalizer=GroupNormalizer(groups=["group_id"], transformation="softplus"),
    add_relative_time_idx=True, add_target_scales=True, add_encoder_length=True,
)

validation = TimeSeriesDataSet.from_dataset(training_dataset, df, predict=True, stop_randomization=True)
batch_size = 64
train_dataloader = training_dataset.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

# --- 4. ОБУЧЕНИЕ (TRAINING) ---
print("\n🧠 ЗАПУСК ОБУЧЕНИЯ (Это займет 3-5 минут)...")

checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1, filename="titan_brain_best")
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=5, verbose=False, mode="min")

trainer = pl.Trainer(
    max_epochs=20,
    accelerator="auto",
    devices=1,
    callbacks=[early_stop_callback, checkpoint_callback],
    enable_model_summary=False,
    gradient_clip_val=0.1,
)

tft = TemporalFusionTransformer.from_dataset(
    training_dataset,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.15,
    hidden_continuous_size=8,
    output_size=7,
    loss=QuantileLoss(),
)

trainer.fit(tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

# --- 5. СИМУЛЯЦИЯ ТОРГОВЛИ ---
print("\n" + "="*60)
print("💰 СИМУЛЯЦИЯ ТОРГОВЛИ (ЗА ПОСЛЕДНИЕ 30 ДНЕЙ)")
print("="*60)

# Загружаем лучший мозг
best_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_path)

# Прогноз
raw_predictions = best_tft.predict(val_dataloader, mode="raw", return_x=True, return_index=True)
preds = raw_predictions.output.prediction
x = raw_predictions.x

def run_simulation(name, profit_target, risk_limit, save_log=False):
    balance = 1000
    trades = []
    wins = 0

    start_idx = preds.shape[0] - BLIND_TEST_HOURS
    if start_idx < 0: start_idx = 0

    for i in range(start_idx, preds.shape[0] - 12):
        log_forecast = preds[i, :, 3].cpu().detach().numpy()[-1]
        log_optimist = preds[i, :, 5].cpu().detach().numpy()[-1]
        log_pessimist = preds[i, :, 1].cpu().detach().numpy()[-1]

        log_entry = x["encoder_target"][i][-1].cpu().detach().numpy()
        entry_price = np.exp(log_entry)
        target_price = np.exp(log_forecast)

        roi = (target_price - entry_price) / entry_price
        risk_spread = (np.exp(log_optimist) - np.exp(log_pessimist)) / target_price

        signal = "NONE"
        if risk_spread <= risk_limit:
            if roi > profit_target: signal = "LONG"
            elif roi < -profit_target: signal = "SHORT"

        if signal != "NONE":
            log_exit = x["decoder_target"][i][-1].cpu().detach().numpy()
            exit_price = np.exp(log_exit)

            if exit_price < 1000: continue

            if signal == "LONG": pnl = (exit_price - entry_price) / entry_price
            else: pnl = (entry_price - exit_price) / entry_price

            net_pnl = pnl - 0.002
            balance *= (1 + net_pnl)

            if net_pnl > 0: wins += 1

            if save_log:
                trades.append({
                    'Bar': i, 'Type': signal, 'Entry': entry_price,
                    'Exit': exit_price, 'PnL_%': round(net_pnl * 100, 2),
                    'Balance': round(balance, 2)
                })

    count = len(trades)
    winrate = (wins/count*100) if count > 0 else 0
    color = "\033[92m" if balance > 1000 else "\033[91m"
    print(f"📊 {name:12} | Риск<{risk_limit*100}% | Сделок: {count:3} | WinRate: {winrate:.1f}% | Итог: {color}${balance:.2f}\033[0m")

    if save_log and count > 0:
        pd.DataFrame(trades).to_csv('titan_trade_log.csv', index=False)
        print(f"   💾 Журнал сделок сохранен: 'titan_trade_log.csv'")

# Проверка стратегий
run_simulation("INVESTOR", 0.006, 0.04)
run_simulation("TRADER",   0.004, 0.06, save_log=True)
run_simulation("SCALPER",  0.003, 0.08)

# --- 6. СОХРАНЕНИЕ МОЗГА ---
print("\n" + "="*50)
print("💾 СОХРАНЕНИЕ ФАЙЛОВ...")
dest_model = "TITAN_BRAIN_RELEASE.ckpt"
shutil.copy(best_path, dest_model)
print(f"   ✅ Мозг: '{dest_model}'")
print(f"   ✅ Логи: 'titan_trade_log.csv'")
print("="*50)

🛠 Установка AI-библиотек (Lightning, Forecasting)...


INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


⚙️ Инициализация TITAN AI...
📉 Всего истории: 9256 часов
✂️ Обучение (Train): До индекса 8583 (Прошлое)
🔒 Тест (Blind):     Последние 720 часов (Скрытое будущее)


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores



🧠 ЗАПУСК ОБУЧЕНИЯ (Это займет 3-5 минут)...


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()


💰 СИМУЛЯЦИЯ ТОРГОВЛИ (ЗА ПОСЛЕДНИЕ 30 ДНЕЙ)


INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


📊 INVESTOR     | Риск<4.0% | Сделок:   0 | WinRate: 0.0% | Итог: $1000.00
📊 TRADER       | Риск<6.0% | Сделок:   0 | WinRate: 0.0% | Итог: $1000.00
📊 SCALPER      | Риск<8.0% | Сделок:   0 | WinRate: 0.0% | Итог: $1000.00

💾 СОХРАНЕНИЕ ФАЙЛОВ...
   ✅ Мозг: 'TITAN_BRAIN_RELEASE.ckpt'
   ✅ Логи: 'titan_trade_log.csv'


In [8]:

# ======================================================
# 🚑 DIAGNOSTIC REBOOT: ПРИНУДИТЕЛЬНЫЙ БЭКТЕСТ
# ======================================================
import torch
import pandas as pd
import numpy as np
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer

print("🔄 ПЕРЕКЛЮЧЕНИЕ В РЕЖИМ ИСТОРИИ (BACKTEST MODE)...")

# 1. Загружаем данные заново (последние 35 дней для запаса)
df_full = pd.read_csv('titan_final_data.csv')
# Исправляем типы
df_full['group_id'] = df_full['group_id'].astype(str)
df_full['hour'] = df_full['hour'].astype(str)
df_full['day'] = df_full['day'].astype(str)
df_full['time_idx'] = df_full['time_idx'].astype(int)

# Берем хвост (30 дней + запас на энкодер 48ч)
backtest_data = df_full.tail(800).copy()

# 2. Создаем датасет с флагом predict=False (ЭТО ГЛАВНОЕ!)
# predict=False заставляет модель идти по истории, а не прыгать в будущее
backtest_ds = TimeSeriesDataSet.from_dataset(
    training_dataset,
    backtest_data,
    predict=False,
    stop_randomization=True
)
backtest_dl = backtest_ds.to_dataloader(train=False, batch_size=128, num_workers=0)

# 3. Загружаем модель
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

# 4. Прогоняем историю
print(f"🧠 Сканирование {len(backtest_data)} часов истории...")
raw_predictions = best_tft.predict(backtest_dl, mode="raw", return_x=True, return_index=True)

preds = raw_predictions.output.prediction
x = raw_predictions.x

print(f"✅ УСПЕХ! Получено прогнозов: {preds.shape[0]}")
print("-" * 75)

# ======================================================
# 🩻 ЗАПУСК РЕНТГЕНА (ТЕПЕРЬ ДАННЫЕ ЕСТЬ)
# ======================================================
print("🔎 АНАЛИЗ ПРИЧИН МОЛЧАНИЯ БОТА...")

total_signals = 0
high_risk_count = 0
missed_profits = 0
risk_log = []

print(f"{'BAR':<6} | {'TYPE':<5} | {'ROI %':<8} | {'RISK %':<10} | {'REAL PNL':<10} | {'RESULT'}")
print("-" * 75)

# Проходим по всем доступным прогнозам
for i in range(preds.shape[0]):
    # Данные
    log_forecast = preds[i, :, 3].cpu().detach().numpy()[-1]
    log_optimist = preds[i, :, 5].cpu().detach().numpy()[-1] # p90
    log_pessimist = preds[i, :, 1].cpu().detach().numpy()[-1] # p10

    # Вход (Encoder Target - последняя точка)
    log_curr = x["encoder_target"][i][-1].cpu().detach().numpy()
    curr = np.exp(log_curr)
    targ = np.exp(log_forecast)

    # Метрики
    roi = (targ - curr) / curr
    # Spread = (Optimist - Pessimist) / Target
    risk_spread = (np.exp(log_optimist) - np.exp(log_pessimist)) / targ

    risk_log.append(risk_spread)

    # Фильтр интереса (смотрим всё, где профит > 0.3%)
    if abs(roi) > 0.003:
        total_signals += 1
        signal = "LONG" if roi > 0 else "SHORT"

        # Реальность (Decoder Target - последняя точка)
        log_exit = x["decoder_target"][i][-1].cpu().detach().numpy()
        exit_p = np.exp(log_exit)

        if exit_p < 1000: continue

        pnl = (exit_p - curr) / curr if signal == "LONG" else (curr - exit_p) / curr
        is_win = pnl > 0.002 # С учетом комиссии

        # Анализ страха (> 8% риск)
        risk_status = ""
        if risk_spread > 0.08:
            high_risk_count += 1
            risk_color = "\033[91m" # Красный
            if is_win:
                missed_profits += 1
                risk_status = "(УПУЩЕНО)"
        else:
            risk_color = "\033[92m" # Зеленый

        # Показываем строки
        if i % 20 == 0 or (is_win and risk_spread > 0.08):
             print(f"{i:<6} | {signal:<5} | {roi*100:>6.2f}%  | {risk_color}{risk_spread*100:>6.2f}%\033[0m   | {pnl*100:>+6.2f}%     | {'✅' if is_win else '❌'} {risk_status}")

# ИТОГ
if len(risk_log) > 0:
    avg_risk = np.mean(risk_log)
    min_risk = np.min(risk_log)

    print("-" * 75)
    print(f"📊 СТАТИСТИКА СТРАХА:")
    print(f"   Средний риск: {avg_risk*100:.2f}%")
    print(f"   Минимальный риск: {min_risk*100:.2f}% <--- ЭТО КЛЮЧЕВАЯ ЦИФРА")
    print(f"   Всего сигналов: {total_signals}")
    print(f"   Отменено из-за страха (>8%): {high_risk_count}")
    print(f"   Упущенные деньги (прибыльные, но страшные): {missed_profits}")

    print("\n💡 РЕКОМЕНДАЦИЯ:")
    if min_risk > 0.08:
        target_risk = min_risk + 0.015
        print(f"   Рынок был очень опасным. Чтобы торговать, установите RISK_LIMIT = {target_risk:.2f}")
    else:
        print(f"   Риски были в норме. Проблема в фильтре ROI. Попробуйте снизить PROFIT_TARGET до 0.003")
else:
    print("⚠️ Странно, массив рисков пуст. Проверьте данные.")

🔄 ПЕРЕКЛЮЧЕНИЕ В РЕЖИМ ИСТОРИИ (BACKTEST MODE)...


INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


🧠 Сканирование 800 часов истории...
✅ УСПЕХ! Получено прогнозов: 741
---------------------------------------------------------------------------
🔎 АНАЛИЗ ПРИЧИН МОЛЧАНИЯ БОТА...
BAR    | TYPE  | ROI %    | RISK %     | REAL PNL   | RESULT
---------------------------------------------------------------------------
0      | LONG  |   0.47%  |   3.19%   |  +0.23%     | ✅ 
20     | LONG  |   0.77%  |   3.05%   |  -0.34%     | ❌ 
80     | SHORT |  -0.53%  |   2.57%   |  -0.46%     | ❌ 
100    | LONG  |   0.60%  |   2.54%   |  +0.45%     | ✅ 
120    | LONG  |   0.66%  |   2.67%   |  -0.23%     | ❌ 
140    | SHORT |  -1.29%  |   3.55%   |  +2.69%     | ✅ 
160    | LONG  |   0.69%  |   3.18%   |  +0.76%     | ✅ 
180    | LONG  |   0.33%  |   3.02%   |  +0.40%     | ✅ 
300    | SHORT |  -1.02%  |   3.64%   |  -1.51%     | ❌ 
320    | SHORT |  -1.81%  |   3.17%   |  +0.19%     | ❌ 
340    | SHORT |  -1.10%  |   3.38%   |  +0.10%     | ❌ 
360    | SHORT |  -0.71%  |   3.17%   |  +1.84%     | ✅ 
4

In [16]:
# ======================================================
# 💎 ТЕСТ "DIAMOND HANDS": РИСК 3.5% (СТРОЖАЙШИЙ)
# ======================================================
import numpy as np

# Жесткий лимит риска
STRICT_RISK = 0.035 # 3.5%

print(f"💎 ЗАПУСК СИМУЛЯЦИИ: МАКСИМАЛЬНЫЙ РИСК {STRICT_RISK*100}%")
print("-" * 75)

def run_strict_test(profit_target):
    balance = 1000
    trades = 0
    wins = 0

    # Проходим по истории (preds уже в памяти после диагностики)
    for i in range(preds.shape[0] - 12):
        # 1. Прогноз
        log_forecast = preds[i, :, 3].cpu().detach().numpy()[-1]
        log_optimist = preds[i, :, 5].cpu().detach().numpy()[-1]
        log_pessimist = preds[i, :, 1].cpu().detach().numpy()[-1]

        log_curr = x["encoder_target"][i][-1].cpu().detach().numpy()
        curr = np.exp(log_curr)
        targ = np.exp(log_forecast)

        # 2. Метрики
        roi = (targ - curr) / curr
        risk_spread = (np.exp(log_optimist) - np.exp(log_pessimist)) / targ

        # 3. Фильтр
        signal = "NONE"
        # УСЛОВИЕ: Риск должен быть МЕНЬШЕ 3.5%
        if risk_spread <= STRICT_RISK:
            if roi > profit_target: signal = "LONG"
            elif roi < -profit_target: signal = "SHORT"

        # 4. Результат
        if signal != "NONE":
            log_exit = x["decoder_target"][i][-1].cpu().detach().numpy()
            exit_p = np.exp(log_exit)

            if exit_p < 1000: continue

            pnl = (exit_p - curr) / curr if signal == "LONG" else (curr - exit_p) / curr
            net = pnl - 0.002

            balance *= (1 + net)
            trades += 1
            if net > 0: wins += 1

    # Вывод
    color = "\033[92m" if balance > 1000 else "\033[91m"
    wr = (wins / trades * 100) if trades > 0 else 0.0

    print(f"Цель прибыли > {profit_target*100:.1f}% | Сделок: {trades:3} | WinRate: {wr:5.1f}% | Итог: {color}${balance:.2f}\033[0m")

# ЗАПУСК ВАРИАНТОВ С РИСКОМ 3.5%
run_strict_test(0.009) # Ваш запрос (0.9% прибыли)
run_strict_test(0.005) # Средняя цель (0.5%)
run_strict_test(0.003) # Скальпинг (0.3%)
run_strict_test(0.001) # Любая прибыль (0.1%)

print("-" * 75)

💎 ЗАПУСК СИМУЛЯЦИИ: МАКСИМАЛЬНЫЙ РИСК 3.5000000000000004%
---------------------------------------------------------------------------
Цель прибыли > 0.9% | Сделок:  95 | WinRate:  47.4% | Итог: $898.18
Цель прибыли > 0.5% | Сделок: 253 | WinRate:  49.8% | Итог: $1375.37
Цель прибыли > 0.3% | Сделок: 380 | WinRate:  43.4% | Итог: $803.85
Цель прибыли > 0.1% | Сделок: 559 | WinRate:  42.2% | Итог: $577.59
---------------------------------------------------------------------------


In [17]:
# ======================================================
# 🩺 АУДИТ ВОЛАТИЛЬНОСТИ: ЖИВ ЛИ БИТКОИН?
# ======================================================
import numpy as np
import pandas as pd

print("📉 ПРОВЕРЯЕМ РЕАЛЬНУЮ АМПЛИТУДУ РЫНКА (ЯНВАРЬ)...")

# Используем данные из памяти (x)
# x["encoder_target"] - это история цен
# x["decoder_target"] - это будущее (через 12 часов)

real_moves = []
abs_moves = []

# Проходим по всем доступным барам
count = x["decoder_target"].shape[0]

for i in range(count):
    # Цена сейчас (последняя точка энкодера)
    log_now = x["encoder_target"][i][-1].cpu().detach().numpy()
    price_now = np.exp(log_now)

    # Цена через 12 часов (последняя точка декодера)
    log_future = x["decoder_target"][i][-1].cpu().detach().numpy()
    price_future = np.exp(log_future)

    # Отсекаем глюки (цена < 1000)
    if price_future < 1000: continue

    # На сколько % цена реально ушла?
    pct_change = (price_future - price_now) / price_now

    real_moves.append(pct_change)
    abs_moves.append(abs(pct_change))

# КОНВЕРТИРУЕМ В ПРОЦЕНТЫ
moves_pct = np.array(abs_moves) * 100

print("-" * 60)
print(f"📊 СТАТИСТИКА ДВИЖЕНИЙ ЗА 12 ЧАСОВ (Real Market Data):")
print("-" * 60)
print(f"   Минимальное движение: {np.min(moves_pct):.2f}%")
print(f"   Среднее движение:     {np.mean(moves_pct):.2f}%  <-- ВАЖНО")
print(f"   Медианное движение:   {np.median(moves_pct):.2f}%")
print(f"   Максимальное движение: {np.max(moves_pct):.2f}%")
print("-" * 60)

# ОЦЕНКА
avg_move = np.mean(moves_pct)
commission_hurdle = 0.2 # 0.2% комиссия

print("⚖️ ВЕРДИКТ:")
if avg_move < 0.5:
    print("   ☠️ РЫНОК МЕРТВ. Среднее движение меньше 0.5%.")
    print("      Торговать с целью 0.9% невозможно физически.")
    print("      Совет: Снижайте таймфрейм или ждите новостей.")
elif avg_move < 1.0:
    print("   ⚠️ РЫНОК ВЯЛЫЙ. Среднее движение около 0.5-1.0%.")
    print("      Цель 0.9% будет срабатывать редко (только на сильных импульсах).")
    print("      Совет: Снизьте PROFIT_TARGET до 0.4-0.5%.")
else:
    print("   ✅ РЫНОК ЖИВОЙ! Среднее движение выше 1.0%.")
    print("      Цель 0.9% абсолютно реальна.")
    print("      Если бот не торгует — проблема в фильтре РИСКА (он боится волатильности).")

📉 ПРОВЕРЯЕМ РЕАЛЬНУЮ АМПЛИТУДУ РЫНКА (ЯНВАРЬ)...
------------------------------------------------------------
📊 СТАТИСТИКА ДВИЖЕНИЙ ЗА 12 ЧАСОВ (Real Market Data):
------------------------------------------------------------
   Минимальное движение: 0.00%
   Среднее движение:     0.80%  <-- ВАЖНО
   Медианное движение:   0.57%
   Максимальное движение: 4.00%
------------------------------------------------------------
⚖️ ВЕРДИКТ:
   ⚠️ РЫНОК ВЯЛЫЙ. Среднее движение около 0.5-1.0%.
      Цель 0.9% будет срабатывать редко (только на сильных импульсах).
      Совет: Снизьте PROFIT_TARGET до 0.4-0.5%.
